# RadioLand Live Coverage Map Demo
### SBE Chapter 15 — Pulling Real Coverage Overlays With Python

In this notebook we'll hit the **RadioLand API** to fetch an FM station's
Longley-Rice coverage prediction and display it on an interactive Folium map —
all in about 20 lines of Python.

The coverage images originate from [RabbitEars.info](https://www.rabbitears.info)
and are cached by the RadioLand backend.

> **How to run:** Click each code cell and press **Shift + Enter** (or the Play button).

---
## Step 1 — Setup

We import our libraries and set the API base URL. Everything here
comes pre-installed on Colab — no `pip install` needed.

In [ ]:
import requests
import re
import base64
import folium
import pandas as pd

# ----------------------------------------------------------------
# RadioLand API base URL  (private server — do not redistribute)
# ----------------------------------------------------------------
API_BASE = 'http://52.151.197.43'

print('Ready!')

---
## Step 2 — Featured Station of the Day

RadioLand picks a different FM station every day as the "Featured Coverage
Map of the Day." One API call gives us the callsign, frequency, city,
format, coordinates, power, and the `app_id` the backend uses to look up
the Longley-Rice coverage file.

In [ ]:
resp = requests.get(f'{API_BASE}/featured_coverage_of_day')
station = resp.json()

# Pretty-print the station info
info = pd.DataFrame([{
    'Callsign':  station['callsign'],
    'Frequency':  f"{station['frequency']} MHz",
    'City':       f"{station['city']}, {station['state']}",
    'Format':     station.get('format', ''),
    'Slogan':     station.get('slogan', ''),
    'ERP (kW)':   station['erp'],
    'Class':      station.get('class_flag', ''),
    'Owner':      station.get('owner', ''),
}])

print(f"Today's featured station: {station['callsign']} {station['frequency']} — "
      f"{station['city']}, {station['state']}")
print()
display(info.T.rename(columns={0: 'Value'}))

---
## Step 3 — Fetch Coverage Image & Map Bounds

Two more API calls:

| Request Type | What It Returns |
|---|---|
| `request_type=7` | The coverage overlay as a **PNG image** |
| `request_type=8` | The geographic **bounding box** (SW and NE corners) |

We download the PNG into memory and **base64-encode** it so Folium can
embed it directly in the map HTML (this avoids browser mixed-content
issues since Colab runs on HTTPS).

In [ ]:
def fetch_coverage(callsign, api_base=API_BASE):
    """
    Fetch a station's coverage overlay image and geographic bounds.

    Returns:
        image_b64  – base64-encoded PNG string (data URL)
        bounds     – [[south, west], [north, east]]  for Folium
    """
    # 1) Coverage image (PNG)
    img_resp = requests.get(f'{api_base}/?request_type=7&callsign={callsign}')
    if img_resp.status_code != 200 or len(img_resp.content) < 100:
        raise ValueError(f'No coverage image found for {callsign}')
    img_b64 = 'data:image/png;base64,' + base64.b64encode(img_resp.content).decode()

    # 2) Map bounds (text: "new GLatLng(lat1,lon1), new GLatLng(lat2,lon2)")
    bnd_resp = requests.get(f'{api_base}/?request_type=8&callsign={callsign}')
    if bnd_resp.status_code != 200:
        raise ValueError(f'No bounds data found for {callsign}')

    match = re.search(
        r'new GLatLng\(([\d.\-]+),\s*([\d.\-]+)\),\s*new GLatLng\(([\d.\-]+),\s*([\d.\-]+)\)',
        bnd_resp.text
    )
    if not match:
        raise ValueError(f'Could not parse bounds for {callsign}: {bnd_resp.text[:120]}')

    lat1, lon1, lat2, lon2 = [float(match.group(i)) for i in range(1, 5)]

    # Folium wants [[south, west], [north, east]]
    south = min(lat1, lat2)
    north = max(lat1, lat2)
    west  = min(lon1, lon2)
    east  = max(lon1, lon2)
    bounds = [[south, west], [north, east]]

    return img_b64, bounds


# Fetch for today's featured station
callsign = station['callsign']
image_data, bounds = fetch_coverage(callsign)

print(f'Coverage image loaded for {callsign}  ({len(image_data):,} chars base64)')
print(f'Bounds: SW {bounds[0]},  NE {bounds[1]}')

---
## Step 4 — Build the Coverage Map

We use **Folium** (a Python wrapper for Leaflet.js) to:
1. Create a base map centered on the station's coordinates
2. Overlay the Longley-Rice coverage PNG within its bounding box
3. Drop a marker on the transmitter site

The coverage shading shows the predicted signal reach — darker areas
indicate stronger signal.

In [ ]:
def build_coverage_map(station_info, image_data, bounds):
    """Build a Folium map with a coverage overlay and transmitter marker."""
    lat = station_info['lat']
    # Station lon may be stored as positive — negate if needed
    lon = -abs(station_info['lon'])

    m = folium.Map(location=[lat, lon], zoom_start=8, tiles='CartoDB positron')

    # Coverage overlay
    folium.raster_layers.ImageOverlay(
        image=image_data,
        bounds=bounds,
        opacity=0.5,
        name='Coverage'
    ).add_to(m)

    # Transmitter marker
    popup_html = (
        f"<b>{station_info['callsign']}</b><br>"
        f"{station_info['frequency']} MHz<br>"
        f"{station_info.get('city', '')}, {station_info.get('state', '')}<br>"
        f"ERP: {station_info['erp']} kW<br>"
        f"Class: {station_info.get('class_flag', '')}<br>"
        f"{station_info.get('slogan', '')}"
    )
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=station_info['callsign'],
        icon=folium.Icon(color='red', icon='signal', prefix='fa')
    ).add_to(m)

    folium.LayerControl().add_to(m)
    return m


m = build_coverage_map(station, image_data, bounds)
m

---
## Step 5 — Try Any Callsign!

Change the callsign below and re-run this cell to see a different
station's coverage. Try your local station!

**Note:** Coverage maps are available for most full-power US FM stations.
Translators (like K201JA) and LPFMs may not have coverage data.

In [ ]:
# ============================================================
#  CHANGE THIS to any FM callsign and re-run the cell!
# ============================================================
MY_CALLSIGN = 'WNJL'

# Fetch station info via callsign search
search_resp = requests.get(
    f'{API_BASE}/?request_type=6&callsign={MY_CALLSIGN}'
)
search_data = search_resp.json()

if 'data' in search_data and len(search_data['data']) > 0:
    stn = search_data['data'][0]
    print(f"{stn.get('callsign', MY_CALLSIGN)} {stn.get('frequency', '')} — "
          f"{stn.get('city', '')}, {stn.get('sp', stn.get('state', ''))}")
    print(f"Format: {stn.get('format', 'N/A')}  |  ERP: {stn.get('erp', 'N/A')} kW")
    print(f"Owner: {stn.get('owner', 'N/A')}")

    # Build station_info dict with the fields our map function expects
    station_info = {
        'callsign':   stn.get('callsign', MY_CALLSIGN),
        'frequency':  stn.get('frequency', ''),
        'city':       stn.get('city', ''),
        'state':      stn.get('sp', stn.get('state', '')),
        'lat':        float(stn.get('lat', 0)),
        'lon':        float(stn.get('lon', 0)),
        'erp':        stn.get('erp', ''),
        'class_flag': stn.get('class_flag', stn.get('class', '')),
        'slogan':     stn.get('slogan', ''),
    }

    try:
        img, bnd = fetch_coverage(MY_CALLSIGN)
        print(f'\nCoverage loaded!  Bounds: SW {bnd[0]}, NE {bnd[1]}')
        build_coverage_map(station_info, img, bnd)
    except ValueError as e:
        print(f'\nNo coverage overlay available: {e}')
else:
    print(f'Station {MY_CALLSIGN} not found. Check the callsign and try again.')

---
### What Just Happened?

With three API calls and ~30 lines of Python, we:

1. **Queried** the RadioLand API for station metadata
2. **Downloaded** a Longley-Rice terrain-based coverage prediction (PNG)
3. **Fetched** the geographic bounding box for that image
4. **Rendered** it on an interactive Leaflet map inside this notebook

The coverage predictions come from [RabbitEars.info](https://www.rabbitears.info)
and are generated using the Longley-Rice Irregular Terrain Model — the same
propagation model used in professional RF engineering tools.

This is the same data that powers the coverage maps on
[radioland.net](https://radioland.net).